In [4]:
#load libraries
import pandas as pd
import os
import numpy as np
from datetime import datetime
from ultralytics import YOLO
from tqdm import tqdm
import cv2

In [5]:
#import images and models
file_path = '/home/lero/idrive/cmac/DDMAP/Stability studies'

#folders = ['Image_analysis_test']
folders = ['40_C_75_RH',
           '40_C_75_RH',
           '30_C_30_RH'
          ]

wells_detection_model_path= '/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/wells_model/Results/fold_3/weights/best.pt'
crystal_detection_model_path = '/home/lero/idrive/cmac/DDMAP/Image_analysis/Code/cryst_amorphous_model/Results/fold_1/weights/best.pt'

wells_detection_model = YOLO(wells_detection_model_path)
crystal_detection_model = YOLO(crystal_detection_model_path)

ignored_classes = {'Label'}

In [6]:
#create functions

# Function to convert filename timestamp to datetime object
def convert_timestamp(filename):
    # ignore extension (last 4 chars)
    return datetime.strptime(filename[:-4], "%Y%m%d_%H%M%S")

def calculate_mean_intensity (image, x1, x2, y1, y2):
    roi = image[y1:y2, x1:x2] #region of interest
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY) #greyscale
    return np.mean(gray_roi.astype(float)) #calculate mean pixel intensity

# Color coding for object classes
class_colors = {
    "Dust": (255, 0, 0),  # Red
    "Amorphous": (0, 255, 0),  # Green
    "Crystal": (255, 255, 0)  # Cyan
}


In [ ]:
#-------run detection model--------#

for folder in folders:
    folder_path = os.path.join(file_path, folder)
    #Image pathway
    for api in os.listdir(folder_path):
        image_folder = os.path.join(folder_path, api)
        if not os.path.isdir(image_folder):
            continue
        print(f'processing {api} in folder {folder}')
        #output folder
        output_folder = os.path.join(image_folder, 'new_results')
        os.makedirs(output_folder, exist_ok=True)

        #get sorted images
        images = sorted(img for img in os.listdir(image_folder) if img.endswith(('.png', '.jpg')))
        if not images:
            print(f'no images found in {image_folder}, skipping')
            continue

        #get the first image timestamp
        assert len(images)
        first_image_timestamp = convert_timestamp(images[0])
        first_image_timestamp_str = first_image_timestamp.strftime("%d-%m-%Y %H:%M")

        # Dictionary to track object states
        object_last_state = {}
        object_detections = {}
        object_positions = []

        wells = list(range(96))
        well_data = {wi: [] for wi in wells}
        stability_results = {wi:{'Timestamp':None, 'Class': None} for wi in wells}
        WINDOW = 20

        #read and process images
        for idx, image_name in enumerate(tqdm(images, unit='images')):
            image_path = os.path.join(image_folder, image_name)
            img = cv2.imread(image_path) #image of well plate

            results = wells_detection_model(img)
            located_wells = []

            for result in results:
                for box in result.boxes:
                    cls_id = int(box.cls)
                    cls_name = result.names[cls_id]
                    conf = round(float(box.conf), 2)

                    if cls_name in ignored_classes:
                        continue

                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                    located_wells.append((x1, y1, x2, y2, center_x, center_y, cls_name, conf)) 
            # --- CHECK-----
            print(f"Image {image_name}: Found {len(located_wells)} wells")
            
            normalised_wells = []
            #normalise grid based on well [0] and map all wells into grid positions
            if located_wells:
                all_confs = sorted([o[-1] for o in located_wells], reverse=True)

                x = [o[4] for o in located_wells if o[-1] > 0.75]
                y = [o[5] for o in located_wells if o[-1] > 0.75]
                left = min(x)
                right = max(x)
                top = min(y)
                bottom = max(y)
                width = right - left
                height = bottom - top
                reference_well = min(located_wells, key=lambda o: (o[5], o[4])) #since origin of image is (0,0)
                ref_x, ref_y = reference_well[4], reference_well[5]

                for obj in located_wells:
                    x1, y1, x2, y2, center_x, center_y, cls_name, conf = obj
                    midp_x = (x1+x2)/2
                    midp_y = (y1+y2)/2
                    x_coord = int((midp_x - left)*12/width/1.03)
                    y_coord = int((midp_y - top)*8/height)
                    x_coord = min(11, max(0, x_coord)) #set min and max rows from 0,11
                    y_coord = min(7, max(0, y_coord)) #set min and max columns from 0,7
                    normalised_wells.append((x1, y1, x2, y2, center_x, center_y, cls_name, conf, x_coord, y_coord))
                
                #keep only the best detections per grid 
                best_detections = {}
                for obj in normalised_wells:
                    x1, y1, x2, y2, center_x, center_y, cls_name, conf, x_coord, y_coord = obj
                    well_key = (x_coord, y_coord)
                    if well_key not in best_detections or conf > best_detections[well_key][7]:
                        best_detections[well_key] = obj
                normalised_wells = list(best_detections.values())


            #-----take patches------#
            for patch in normalised_wells:
                x1, y1, x2, y2, center_x, center_y, cls_name, conf, x_coord, y_coord = patch
                well_key = y_coord*12+x_coord #0-95 indexing
                #crop the patch using the coordinates from grid position
                well_crop = img[y1:y2, x1:x2]
                ##--possible to save well image here--##

                crystal_results = crystal_detection_model(well_crop)
                
                #extract top classification prediction
                top_cls_name=None
                top_conf= 0
                for res in crystal_results:
                    for crop in res.boxes:
                        patch_cls_id = int(crop.cls)
                        patch_conf = float(crop.conf)
                        if patch_conf > top_conf:
                            top_conf = patch_conf
                            top_cls_name = res.names[patch_cls_id]
                            
                #store result
                well_data[well_key].append({
                    "image": image_name,
                    "crop_coords": (x1, y1, x2, y2),
                    "class": top_cls_name,
                    "conf": top_conf
                })

            #----annotate every 50th image for a visual check----#
            if idx % 50 == 0 and normalised_wells:
                annotated_img = img.copy()
                for patch_info in normalised_wells:
                    x1, y1, x2, y2, center_x, center_y, cls_name, conf, x_coord, y_coord = patch_info
                    well_key = y_coord * 12 + x_coord  # 0-95 index
            
                    if well_data.get(well_key):
                        top_result = well_data[well_key][-1]  # last classification for this image
                        label = f"{top_result['class']}, ({top_result['conf']:.2f})"
            
                        # Draw rectangle and label
                        color = class_colors.get(top_result['class'], (255, 255, 255))  # default white
                        cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
                        cv2.putText(annotated_img, label, (x1, y1 - 5),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
                annotated_path = os.path.join(output_folder, f"annotated_{idx:04d}_{image_name}")
                cv2.imwrite(annotated_path, annotated_img)
                print(f"✅ Saved annotated frame #{idx}: {annotated_path}")


            #----------- calculate well stability over time ----------#
            for well_key in wells:
                entries = well_data[well_key]
                if stability_results[well_key]['Class'] is None and len(entries) >= WINDOW:
                    for i in range(len(entries) - WINDOW + 1):
                        current_class = entries[i]["class"]
                        if current_class == 'Crystal':
                            next_classes = [e['class'] for e in entries[i:i+WINDOW]]
    
                            if all(c=='Crystal' for c in next_classes):
                                stability_results[well_key]["Timestamp"] = entries[i]['image']
                                stability_results[well_key]['Class'] = 'Unstable'
                                break

        
        for well_key, entries in well_data.items():
            rows = []
            for entry in entries:
                rows.append({
                    "well": well_key,
                    "timestamp": entry["image"],
                    "class": entry["class"]
                })
        
            well_df = pd.DataFrame(rows)
            well_df_path = os.path.join(output_folder, f"well_{well_key}_data.csv")
            well_df.to_csv(well_df_path, index=False)
            print("well data saves as csv")
        
        stability_rows = []
        for well_key, info in stability_results.items():
            stability_rows.append({
                "well": well_key,
                "unstable_timestamp": info["Timestamp"],
                "status": info["Class"] if info["Class"] else "Stable"
            })
        
        stability_df = pd.DataFrame(stability_rows)
        
        stability_df_path = os.path.join(output_folder, "stability_results.csv")
        stability_df.to_csv(stability_df_path, index=False)
        print("✅ stability_results saved as CSV")

print('hopefully done')                                                        

processing FEL in folder 40_C_75_RH


  0%|                                                                                                                                                          | 0/752 [00:00<?, ?images/s]/home/lero/LeRo_venv/lib/python3.13/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")



0: 576x1024 98 Wells, 1 Label, 57.1ms
Speed: 6.1ms preprocess, 57.1ms inference, 128.7ms postprocess per image at shape (1, 3, 576, 1024)
Image 20240821_132027.jpg: Found 98 wells

0: 256x256 1 Amorphous, 4.8ms
Speed: 0.5ms preprocess, 4.8ms inference, 1.2ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 5.1ms
Speed: 0.4ms preprocess, 5.1ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 4.8ms
Speed: 0.4ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.8ms
Speed: 0.4ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.8ms
Speed: 0.4ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.8ms
Speed: 0.4ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.8ms
Speed: 0.4ms preprocess, 4.8ms inference, 0.9ms pos

  0%|▏                                                                                                                                                 | 1/752 [00:01<23:20,  1.86s/images]

✅ Saved annotated frame #0: /home/lero/idrive/cmac/DDMAP/Stability studies/40_C_75_RH/FEL/new_results/annotated_0000_20240821_132027.jpg

0: 576x1024 99 Wells, 1 Label, 7.0ms
Speed: 4.1ms preprocess, 7.0ms inference, 25.9ms postprocess per image at shape (1, 3, 576, 1024)
Image 20240821_152121.jpg: Found 99 wells

0: 256x256 1 Dust, 5.2ms
Speed: 0.5ms preprocess, 5.2ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 5.0ms
Speed: 0.4ms preprocess, 5.0ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 4.8ms
Speed: 0.3ms preprocess, 4.8ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.7ms
Speed: 0.4ms preprocess, 4.7ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 1 Dust, 4.7ms
Speed: 0.3ms preprocess, 4.7ms inference, 1.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 4.8ms
Speed: 0.4ms preproce

...


0: 576x1024 101 Wells, 1 Label, 7.0ms
Speed: 4.1ms preprocess, 7.0ms inference, 27.4ms postprocess per image at shape (1, 3, 576, 1024)
Image 20240824_022725.png: Found 101 wells

0: 256x256 1 Dust, 5.4ms
Speed: 0.5ms preprocess, 5.4ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 5.5ms
Speed: 0.5ms preprocess, 5.5ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 5.0ms
Speed: 0.5ms preprocess, 5.0ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 4.8ms
Speed: 0.4ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 5.0ms
Speed: 0.4ms preprocess, 5.0ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.9ms
Speed: 0.4ms preprocess, 4.9ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.9ms
Speed: 0.5ms preprocess, 4.9ms inference, 0.9ms post

  3%|████▋                                                                                                                                            | 24/752 [00:27<14:00,  1.16s/images]


0: 576x1024 101 Wells, 1 Label, 7.0ms
Speed: 4.0ms preprocess, 7.0ms inference, 26.4ms postprocess per image at shape (1, 3, 576, 1024)
Image 20240824_042929.png: Found 101 wells

0: 256x256 1 Dust, 5.1ms
Speed: 0.5ms preprocess, 5.1ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 1 Dust, 4.9ms
Speed: 0.5ms preprocess, 4.9ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 4.7ms
Speed: 0.5ms preprocess, 4.7ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.8ms
Speed: 0.4ms preprocess, 4.8ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.7ms
Speed: 0.4ms preprocess, 4.7ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.7ms
Speed: 0.5ms preprocess, 4.7ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.8ms
Speed: 0.4ms preprocess, 4.8ms inference, 0.

  3%|████▊                                                                                                                                            | 25/752 [00:28<13:34,  1.12s/images]


0: 576x1024 101 Wells, 1 Label, 7.0ms
Speed: 4.3ms preprocess, 7.0ms inference, 26.3ms postprocess per image at shape (1, 3, 576, 1024)
Image 20240824_063133.png: Found 101 wells

0: 256x256 1 Dust, 5.2ms
Speed: 0.5ms preprocess, 5.2ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 5.1ms
Speed: 0.4ms preprocess, 5.1ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.9ms
Speed: 0.4ms preprocess, 4.9ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 5.0ms
Speed: 0.4ms preprocess, 5.0ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.9ms
Speed: 0.4ms preprocess, 4.9ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Amorphous, 4.9ms
Speed: 0.4ms preprocess, 4.9ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Dust, 4.9ms
Speed: 0.4ms preprocess, 4.9ms inference, 0.9ms post